In [36]:
:opt no-lint

# Chapter 13. Building projects
## 13.1 Modules
Haskell programs are organized into modules. Modules contain the datatypes, type synonyms, typeclasses, typeclass instances, and values you’ve defined at the top level.
This chapter covers:
* writing Haskell programs with modules;
* using the Cabal package manager;
* building our project with Stack;
* conventions around project organization;
* building a small interactive game.

## 13.2 Making packages with Stack
* The Haskell Cabal is a package manager.
* Stack is a cross-platform program for developing Haskell projects.
* Stack is built on top of Cabal.
* Stack simplifies the process.
* Stack elies on an LTS snapshot of Haskell packages from Stackage.

## 13.3 Working with a basic project
* `stack build` builds the project.
* `stack setup` installs the right LTS version of GHC.
* `stack ghci` starts the GHCi REPL
* `stack exec` run the built project

The executable is defined by the `.cabal` file.
## 13.4 Making our project a library
A module can be defined as a library and added to the dependencies by adding its source path to `build-depends` entry.
## 13.5 Module exports
By default every top-level binding is exported. For larger modules is better to specify the exports.
## 13.6 More on importing modules
* Imported modules are top-level declarations.
* The entities imported as part of those declarations have scope throughout the module, and they can be shadowed by local bindings. 
* The ordering of import declarations is irrelevant.

The content of a module can be explored with `:browse`
## 13.7 Making our program interactive
Use the `do` syntax and `getLint` function
## 13.8 `do` syntax and `IO`
The `do` syntax is a syntactic sugar that allows to sequence _monadic_ actions.
Is better to use `>>=` in single-line expressions instead of `do`.
## 13.9 Hangman game
Command `stack new` create a new project.
## 13.14 Chapter exercises
Changing the game so that, as with normal hangman, only incorrect guesses count towards the guess limit.

In [37]:
import Control.Monad (forever)
import Data.Char (toLower)
import Data.Maybe (isJust)
import Data.List (intersperse, nub)
import System.Exit (exitSuccess)
import System.Random (randomRIO)
import System.IO

type WordList = [String]

allWords :: IO WordList
allWords = do
  dict <- readFile "data/dict.txt"
  return (lines dict)

minWordLength :: Int
minWordLength = 5

maxWordLength :: Int
maxWordLength = 10

gameWords :: IO WordList
gameWords = do
  aw <- allWords
  return (filter gameLength aw)
  where gameLength w =
          let l = length (w :: String)
          in  l > minWordLength && l < maxWordLength

randomWord :: WordList -> IO String
randomWord wl = do
  randomIndex <- randomRIO (0, length wl - 1)
  return $ wl !! randomIndex

randomWord' :: IO String
randomWord' = gameWords >>= randomWord

data Puzzle = Puzzle String [Maybe Char] [Char]

instance Show Puzzle where
  show (Puzzle _ discovered guessed) =
    (intersperse ' ' $ fmap renderPuzzleChar discovered)
    ++ " Guessed so far: " ++ guessed

freshPuzzle :: String -> Puzzle
freshPuzzle word = Puzzle word discovered guessed
              where discovered = map (const Nothing) word
                    guessed = []

charInWord :: Puzzle -> Char -> Bool
charInWord (Puzzle word _ _) c = elem c word

alreadyGuessed :: Puzzle -> Char -> Bool
alreadyGuessed (Puzzle _ _ guessed) c = elem c guessed

renderPuzzleChar :: Maybe Char -> Char
renderPuzzleChar Nothing = '_'
renderPuzzleChar (Just c) = c

fillInCharacter :: Puzzle -> Char -> Puzzle
fillInCharacter (Puzzle word discovered guessed) c =
  Puzzle word newDiscovered (c : guessed)
  where zipper guess wordChar discoveredChar =
          if wordChar == guess
          then Just wordChar
          else discoveredChar
        newDiscovered =
          zipWith (zipper c) word discovered

handleGuess :: Puzzle -> Char -> IO Puzzle
handleGuess puzzle guess = do
  putStrLn $ "Your guess was: " ++ [guess]
  case (charInWord puzzle guess, alreadyGuessed puzzle guess) of
    (_, True) -> do
      putStrLn "You already guessed that, pick something else!"
      return puzzle
    (True, _) -> do
      putStrLn "Great guess! That's in, filling it accordingly."
      return (fillInCharacter puzzle guess)
    (False, _) -> do
      putStrLn "Too bad! This isn't in the word, try again."
      return (fillInCharacter puzzle guess)

gameOver :: Puzzle -> IO ()
gameOver (Puzzle wordToGuess filledInSoFar guessed)
  if (length guessed - (length $ nub $ filter isJust filledInSoFar) > 7) then
    do putStrLn "You lost!"
       putStrLn $ "The answer was: " ++ wordToGuess
       exitSuccess
  else
    return ()

gameWin :: Puzzle -> IO ()
gameWin (Puzzle _ filledInSoFar _) =
  if all isJust filledInSoFar then
    do putStrLn "You win!"
       exitSuccess
  else return ()

runGame :: Puzzle -> IO ()
runGame puzzle = forever $ do
  gameOver puzzle
  gameWin puzzle
  putStrLn $ "Current puzzle is: " ++ show puzzle
  putStr "Guess a letter: "
  guess <- getLine
  case guess of
    [c] -> handleGuess puzzle c >>= runGame
    _   -> putStrLn "Your guess must be a single character."

main :: IO ()
main = do
  word <- randomWord'
  let puzzle = freshPuzzle (fmap toLower word)
  runGame puzzle

: 

1. Ciphers: Open your `Ciphers` module and modify it so that the Caesar and Vigenère ciphers work with user input.

In [38]:
import Data.Char (chr, ord)


type PlainText = String
type CipherText = String


vigenère :: String -> PlainText -> CipherText
vigenère secret = zipWith (shift (+)) (cycle secret) . concat . words

unvigenère :: String -> CipherText -> PlainText
unvigenère secret = zipWith (shift (-)) (cycle secret) . concat . words

shift :: (Int -> Int -> Int) -> Char -> Char -> Char
shift op offset ch = numToChar $ charToNum ch `op` charToNum offset
  where
    charToNum ch = ord ch - ord 'A'
    numToChar n = chr $ (n `mod` 26) + ord 'A'


cæsar :: Int -> PlainText -> CipherText
cæsar secret = map
  (((chr . (\x -> if x > lastOrd then x - 26 else x)) . (+secret)) . ord)
  
uncæsar :: Int -> CipherText -> PlainText
uncæsar secret = map
  (((chr . (\x -> if x < firstOrd then x + 26 else x)) . (\x -> x - secret)) . ord)

firstOrd = ord 'a'
lastOrd = firstOrd + 26


run :: String -> String -> String -> String
run operation secret text
  | operation == "cæsar" = cæsar (read secret :: Int) text
  | operation == "uncæsar" = uncæsar (read secret :: Int) text
  | operation == "vigenère" = vigenère secret text
  | operation == "unvigenère" = unvigenère secret text
  | otherwise = "Invalid operation"

main = do
  putStr "Enter the text: "
  text <- getLine
  putStr "Enter the key: "
  secret <- getLine
  putStr "Select the operation: "
  operation <- getLine
  putStrLn $ run operation secret text

2. Here is a very simple, short block of code. Notice it has a forever that will make it keep running, over and over again. Load it into your REPL and test it out. Then refer back to the chapter and modify it to exit successfully after a False result.

In [39]:
import Control.Monad
import System.Exit (exitSuccess)

palindrome :: IO ()
palindrome = forever $ do
  line1 <- getLine
  case (line1 == reverse line1) of
    True -> putStrLn "It's a palindrome!"
    False -> do
      putStrLn "Nope!"
      exitSuccess

3. If you tried using palindrome on a sentence such as “Madam I’m Adam,” you may have noticed that palindrome checker doesn’t work on that. Modifyg the above so that it works on sentences too.

In [40]:
import Control.Monad
import Data.Char (isLetter, toLower)

palindrome :: IO ()
palindrome = forever $ do
  line1 <- getLine
  let normalized = map toLower $ filter isLetter line1
  case (normalized == reverse normalized) of
    True -> putStrLn "It's a palindrome!"
    False -> putStrLn "Nope!"

4.

In [41]:
import Data.Char
import System.IO

type Name = String
type Age = Integer

data Person = Person Name Age deriving Show

data PersonInvalid = NameEmpty | AgeTooLow | PersonInvalidUnknown String deriving (Eq, Show)

mkPerson :: Name -> Age -> Either PersonInvalid Person
mkPerson name age
  | name /= "" && age > 0 = Right $ Person name age
  | name == ""            = Left NameEmpty
  | not (age > 0)         = Left AgeTooLow
  | otherwise             = Left $ PersonInvalidUnknown $
                                     "Name was: " ++ show name ++
                                     " Age was: " ++ show age

gimmePerson :: IO ()
gimmePerson = do
  putStr "Enter name: "
  name <- getLine
  putStr "Enter age: "
  age <- getLine
  let person = mkPerson name $ read age
  case person of
    Right x -> putStrLn $ show x
    Left e -> putStrLn $ show e